In [2]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import cvxpy as cp
from math import ceil

data = pd.read_excel('data.xlsx', header=None).values.flatten()

In [3]:
requirements = data
requirements

array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
         5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
         9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
        10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
        50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
        41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
        54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
        89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
      dtype=int64)

In [4]:
total_week = 104
requirements = requirements[0:total_week]
K = 20
waste_rate = 0.1

In [5]:
working_hands = cp.Parameter(total_week)
waste_hands = cp.Parameter(total_week)
working_hands.value = 4 * requirements
waste_hands.value = np.array([
    round(4 * x * waste_rate) if x % 1 != 0.5 else ceil(4 * x * waste_rate) 
    for x in requirements
])


teaching_hands = cp.Variable(total_week, integer=True)
resting_hands = cp.Variable(total_week, integer=True)
new_hands = cp.Variable(total_week, integer=True)

In [6]:
working_hands.value, waste_hands.value

(array([ 44,  20,  16,  28,  64,  24,  20,  28,  52,  24,  20,  28,  48,
         20,  16,  24,  36,  20,  20,  44, 116,  84,  68,  80, 108,  52,
         36,  40,  64,  24,  20,  28,  44,  20,  20,  24,  48,  28,  28,
         40,  60,  40,  36,  44,  60,  40,  40,  64, 104,  84,  92, 144,
        200, 180, 180, 196, 228, 172, 160, 176, 208, 172, 168, 180, 208,
        164, 156, 164, 192, 140, 136, 140, 168, 136, 144, 172, 220, 192,
        216, 260, 320, 280, 296, 340, 404, 356, 352, 360, 400, 348, 352,
        356, 416, 356, 356, 360, 424, 384, 376, 396, 436, 396, 384, 408],
       dtype=int64),
 array([ 4,  2,  2,  3,  6,  2,  2,  3,  5,  2,  2,  3,  5,  2,  2,  2,  4,
         2,  2,  4, 12,  8,  7,  8, 11,  5,  4,  4,  6,  2,  2,  3,  4,  2,
         2,  2,  5,  3,  3,  4,  6,  4,  4,  4,  6,  4,  4,  6, 10,  8,  9,
        14, 20, 18, 18, 20, 23, 17, 16, 18, 21, 17, 17, 18, 21, 16, 16, 16,
        19, 14, 14, 14, 17, 14, 14, 17, 22, 19, 22, 26, 32, 28, 30, 34, 40,
        36, 35

In [7]:
cons = [
    resting_hands[0] + teaching_hands[0] + working_hands[0] == 50,
    K * teaching_hands >= new_hands,
    teaching_hands >= 0,
    resting_hands >= 0,
    new_hands >= 0
]

In [8]:
for t in range(total_week - 1):
    cons.append(resting_hands[t + 1] >= working_hands[t] - waste_hands[t]),
    cons.append(resting_hands[t + 1] + working_hands[t + 1] + teaching_hands[t + 1] == \
        resting_hands[t] + working_hands[t] + teaching_hands[t] + new_hands[t] - waste_hands[t]
        )

In [9]:
obj = cp.Minimize(
    100 * cp.sum(new_hands)\
    + 5 * cp.sum(resting_hands)\
    + 10 * cp.sum(teaching_hands + new_hands)
)

In [10]:
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.CPLEX, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 04 06:31:19 AM: Your problem has 312 variables, 211 constraints, and 208 parameters.
(CVXPY) May 04 06:31:19 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 04 06:31:19 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 04 06:31:19 AM: Compiling problem (target solver=CPLEX).
(CVXPY) May 04 06:31:19 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> CPLEX
(CVXPY) May 04 06:31:19 AM: Applying reduction CvxAttr2Constr
(CVXPY) May 04 06:31:19 AM: Applying reduction Qp2Symb

333715.0

In [11]:
working_hands.value, resting_hands.value

(array([ 44,  20,  16,  28,  64,  24,  20,  28,  52,  24,  20,  28,  48,
         20,  16,  24,  36,  20,  20,  44, 116,  84,  68,  80, 108,  52,
         36,  40,  64,  24,  20,  28,  44,  20,  20,  24,  48,  28,  28,
         40,  60,  40,  36,  44,  60,  40,  40,  64, 104,  84,  92, 144,
        200, 180, 180, 196, 228, 172, 160, 176, 208, 172, 168, 180, 208,
        164, 156, 164, 192, 140, 136, 140, 168, 136, 144, 172, 220, 192,
        216, 260, 320, 280, 296, 340, 404, 356, 352, 360, 400, 348, 352,
        356, 416, 356, 356, 360, 424, 384, 376, 396, 436, 396, 384, 408],
       dtype=int64),
 array([  5.,  40.,  42.,  26.,  25.,  59.,  61.,  50.,  25.,  48.,  50.,
         39.,  25.,  48.,  50.,  40.,  26.,  38.,  35.,  18.,  40., 104.,
        112.,  92.,  72., 117., 128., 120.,  92., 126., 128., 118.,  99.,
        119., 117., 111.,  85., 100.,  97.,  82.,  58.,  72.,  72.,  60.,
         40.,  54.,  49.,  36.,  58.,  94.,  76.,  83., 130., 180., 162.,
        162., 176., 209.

In [12]:
new_hands.value, np.array(teaching_hands.value, dtype=np.int32)

(array([ 14.,   0.,   0.,  36.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          8.,   0.,   0.,   0.,   0.,   0.,   0.,  13.,  96.,  41.,   0.,
          0.,  15.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,  18.,  66.,  25.,   1.,  70., 114.,  48.,   1.,  36.,
         62.,   0.,   0.,   0.,  37.,  13.,   0.,   9.,  55.,   1.,   0.,
          0.,  37.,   0.,   0.,   0.,  16.,   9.,   0.,  48.,  87.,  37.,
         21.,  89., 122.,  44.,  12.,  90., 134.,  47.,   0.,  33.,  80.,
         22.,   0.,  40.,  96.,  34.,   0.,  28., 101.,  57.,   0.,  49.,
         95.,  38.,   0.,  44.,  -0.]),
 array([1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 5, 3, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 4, 2, 1, 4, 6, 3, 1, 2, 4, 0, 0, 0, 2, 1, 0, 1, 3, 1, 0,
        0, 2, 0, 0, 0, 1, 1, 0, 3, 5, 2, 2, 5, 7, 3, 1, 5, 7, 3, 0, 2, 4